In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
%matplotlib inline

In [2]:
def swc_to_dataframe( file_name ) :
    col_names = ['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level', 'mode', 'timestamp', 'TFresindex']
    df = pd.read_csv( file_name, sep=r"\s+", comment = "#", names = col_names, index_col = 'id')
    return df

In [3]:
def filter_swc( df, type ) :
    dd = df[df['type']==type]
    return dd

In [4]:
model_directory = "/allen/programs/celltypes/workgroups/mousecelltypes/_UPENN_fMOST"
swc_directory = os.path.join( model_directory,  "SWC_files", "SWCfile_67cells(Allen&WMU)_original")
image_directory = os.path.join( model_directory, "Inputs", "Original_Downsampled20um")

output_directory = os.path.join( model_directory, "SWC_files", "SWCfile_67cells(Allen&WMU)_original_thumbnail")

In [5]:
#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}

#
# views for swc
#
views = {}
views['coronal'] = {'order': 0, 'x': 'y', 'y': 'x'}
views['sagittal'] = {'order': 1, 'x': 'z', 'y' : 'x'}
views['horizontal'] = {'order': 2, 'x': 'z', 'y' : 'y'}

#
# image slices
#
slices = {}
slices['coronal'] = {'swc': 'z', 'image' : 'z', 'transpose' : False }
slices['sagittal'] = {'swc': 'y', 'image' : 'x', 'transpose' : True }
slices['horizontal'] = {'swc' : 'x', 'image' : 'y', 'transpose' : True }

#
# scale from image pixel index to SWC coordinate
#
scales = {}
scales['z'] = 16
scales['x'] = 64
scales['y'] = 64

In [6]:
#
# Create morphology thumbnail for each SWC file in the swc directory
#
#
for swc_file in glob.glob(os.path.join(swc_directory, "*.swc")):
    
    bb = os.path.basename( swc_file )
    
    # find the associated brain file
    bname = bb.split('_')[0]
    image_file = os.path.join(image_directory, "mouse%sred_xy32z8.nii.gz" % bname )
    
    if not os.path.isfile( image_file ) :
        continue
    
    # set the output thumbnail png file name
    cname = bb.split('.')[0]
    thumnail_file = os.path.join( output_directory, "%s.png" % cname )
    
    print((cname,bname))
    
    #
    # read swc components into dataframe
    #
    df = swc_to_dataframe( swc_file )

    #
    # read in image file
    #
    image = sitk.ReadImage( image_file )
    print(image.GetSize())


    #
    # scale swc coordinates to match image volume
    #
    for sk, sv in scales.items() :
        df[sk] = df[sk] / sv


    #
    # get background image from soma location
    #

    filtered = filter_swc(df,types['soma']['type_id'])
    
    if len(filtered) == 0 :
        print("no soma found")
        continue
        
    soma = filtered.iloc[0]


    #
    #  plot swc components onto of image at soma location
    #
    plt.rcParams["figure.figsize"] = (30,8)

    fig, axs = plt.subplots(1,3)
    fig.suptitle(os.path.basename(swc_file))

    for vk, vv in views.items() :

        idx = int(round(soma[slices[vk]['swc']],0))

        if slices[vk]['image'] == 'x' :
            bkg = sitk.GetArrayViewFromImage( image )[:,:,idx]
        elif slices[vk]['image'] == 'y' :
            bkg = sitk.GetArrayViewFromImage( image )[:,idx,:]
        elif slices[vk]['image'] == 'z' :
            bkg = sitk.GetArrayViewFromImage( image )[idx,:,:]

        if slices[vk]['transpose'] :
            bkg = bkg.transpose()

        axs[vv['order']].imshow( bkg, cmap='gray')

        for tk,tv in types.items() :

            filtered = filter_swc(df, tv['type_id'])

            x = filtered[vv['x']].values
            y = filtered[vv['y']].values

            axs[vv['order']].scatter( x, y, s=0.001, c=tv['color'])

        axs[vv['order']].set_aspect('equal')

    
    plt.savefig(thumnail_file, dpi = 150)
    plt.close("all")
    

('191807_5062-X6404-Y23808', '191807')
(481, 325, 677)
('191812_4003-X5886-Y22658', '191812')
(481, 295, 729)
('182724_2525-X8483-Y21088', '182724')
(481, 297, 720)
('191812_3922-X8455-Y6768', '191812')
(481, 295, 729)
('191812_3915-X8011-Y24180', '191812')
(481, 295, 729)
('191807_5629-X4661-Y9410', '191807')
(481, 325, 677)
('182724_2383-X9300-Y17890', '182724')
(481, 297, 720)
no soma found
('191812_5586-X4875-Y23378', '191812')
(481, 295, 729)
('191807_5178-X5002-Y23078', '191807')
(481, 325, 677)
('191812_5411-X6011-Y24641', '191812')
(481, 295, 729)
('191812_5922-X4572-Y8645', '191812')
(481, 295, 729)
('191807_4923-X7499-Y24194', '191807')
(481, 325, 677)
('191812_4775-X5212-Y22490', '191812')
(481, 295, 729)
('182724_4599-X8181-Y6776', '182724')
(481, 297, 720)
('191812_6351-X5009-Y7424', '191812')
(481, 295, 729)
('191812_5219-X4127-Y21100', '191812')
(481, 295, 729)
('191812_3758-X8791-Y24162', '191812')
(481, 295, 729)
('182724_4472-X5247-Y12162', '182724')
(481, 297, 720)
(